In [197]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [273]:

PID = pd.read_csv('Compiled/PID.csv',index_col='id_victim',parse_dates=['date'])
Updated = pd.read_csv('Compiled/UpdatedLocations.csv',index_col='id_victim',parse_dates=['date'])
# Updated = Updated[Updated.index.duplicated()==False]
PID['extra_source']=''
if Updated.loc[Updated.index.isin(PID.index)==False].shape[0]>=1:
    print('Changed/Missing: ')
    print(Updated.loc[Updated.index.isin(PID.index)==False].index)
    Updated=Updated.drop(Updated.loc[Updated.index.isin(PID.index)==False].index)
print(Updated.shape)
print(PID.loc[PID.index.isin(Updated.index)].shape)
PID.loc[PID.index.isin(Updated.index)]=Updated
print(PID.groupby('Check').count()['name'])

# print(Updated)
# for i,row in Updated.index)

PID = PID.reset_index(drop=False).set_index(pd.DatetimeIndex(PID.date))


prov = 'BC'
BC = PID.loc[((PID.prov==prov))].sort_index(ascending=False)

print()
print('Prov: '+ prov+', Total: ',BC.shape[0])
Missing = BC.loc[BC['Check']=='Yes']

print('Tasks Remaining: ',Missing.shape[0])
Missing[['name','id_victim','city_town','race','summary']].head()

(62, 41)
(62, 41)
Check
Added       59
Ignore       3
No         462
Yes       1138
Name: name, dtype: int64

Prov: BC, Total:  311
Tasks Remaining:  151


,name,id_victim,city_town,race,summary
date,,,,,
2016-02-25,William Ryan Fisher,152_GS,Maple Ridge,NaN,Observed to have ingested drugs while incarcer...
2016-02-21,Unspecified,151_GS,Smithers,NaN,Lost consciousness in RCMP custody on Feb 15; ...
2015-12-26,Gurwinder Singh Mann,150_GS,Abbotsford,NaN,Accidental; TBD; Federal corrections; Federal ...
2015-12-16,Unspecified,149_GS,Surrey,NaN,TBD; Coroner's Service yet to announce investi...
2015-09-23,Unspecified,148_GS,Agassiz,NaN,Self-inflicted injury while being apprehended ...


In [265]:
def Entries():
    city = input('Enter City/Town')
    address = input('Enter Address/Intersection')
    postal_code = input('Enter Postal Code')
    return(city,address,postal_code)
Info = ['name','age','race','gender','cause_death','date','data_source','city_town','address_intersection','postal_code','department']
for i, row in Missing[0:5].iterrows():
    print(row[Info])
#     print(row['name','age','race','Date','data_source','department'])
#     print(row['city_town'])
    print(row['summary'])
    Skip = input('skip/add source/ignore: s/a/i')
    if Skip != 's' and Skip != 'i':
        if Skip == 'a':
            text = input('Enter Source')
            Missing.loc[Missing['id_victim']==row['id_victim'],'extra_source']=text
            text = input('Enter Summary')
            Missing.loc[Missing['id_victim']==row['id_victim'],'summary']+='     '+text
            text = input('Enter Name')
            if text != '':
                Missing.loc[Missing['id_victim']==row['id_victim'],'name']=text
            text = input('Enter Race')
            if text != '':
                Missing.loc[Missing['id_victim']==row['id_victim'],'race']=text
            text = input('Enter Gender')
            if text != '':
                Missing.loc[Missing['id_victim']==row['id_victim'],'gender']=text
            text = input('Enter Department')
            if text != '':
                Missing.loc[Missing['id_victim']==row['id_victim'],'department']=text
            text = input('Enter cause_death')
            if text != '':
                Missing.loc[Missing['id_victim']==row['id_victim'],'cause_death']=text
                
                
            clear_output()
            print(Missing.loc[Missing.id_victim==row['id_victim'],
                  Info].values)
            print(Missing.loc[Missing.id_victim==row['id_victim'],
                  ['summary']].values)
            
        city,address,postal_code=Entries()
        print('Entries: ',city,address,postal_code)
        keep = input('Skip/Repeat: r/s')
       
        if keep == 'r':
            city,address,postal_code=Entries()
            print('Entries: ',city,address,postal_code)
            keep = input('Skip: s')
            if keep != 's':
                Missing.loc[Missing['id_victim']==row['id_victim'],'city_town']=city
                Missing.loc[Missing['id_victim']==row['id_victim'],'address_intersection']=address
                Missing.loc[Missing['id_victim']==row['id_victim'],'postal_code']=postal_code
                Missing.loc[Missing['id_victim']==row['id_victim'],'Check']='Added'
        elif keep != 's':
            Missing.loc[Missing['id_victim']==row['id_victim'],'city_town']=city
            Missing.loc[Missing['id_victim']==row['id_victim'],'address_intersection']=address
            Missing.loc[Missing['id_victim']==row['id_victim'],'postal_code']=postal_code
            Missing.loc[Missing['id_victim']==row['id_victim'],'Check']='Added'
    if Skip == 'i':
        print('ignore')
#         print(row['id_victim'])
        Missing.loc[Missing['id_victim']==row['id_victim'],'Check']='Ignore'
#         print(Missing.loc[Missing['id_victim']==row['id_victim'],'Check'])
    clear_output()
    #     Unspecified
# print(Missing['Check'])
# print(Missing.loc[((Missing['Check']=='Added')|
#                  (Missing['Check']=='Ignore'))])
# print(Missing)
Missing = Missing.reset_index(drop=True).set_index(Missing['id_victim']).drop('id_victim',axis=1)
Updated = Updated.append(Missing.loc[((Missing['Check']=='Added')|
                 (Missing['Check']=='Ignore'))].copy())
Updated.to_csv('Compiled/UpdatedLocations.csv')

name                            Unspecified
age                                     NaN
race                                    NaN
gender                                 male
cause_death                             TBD
date                    2016-02-21 00:00:00
data_source               Georgia St (2017)
city_town                          Smithers
address_intersection                    NaN
postal_code                     Unspecified
department                    RCMP Smithers
Name: 2016-02-21 00:00:00, dtype: object
Lost consciousness in RCMP custody on Feb 15; died in hospital Feb 21.; Coroner's Service yet to announce investigation; TBD; TBD; TBD       **<p>British Columbia is in the midst of a crisis involving police killings of civilians over the period of several years. The Independent Investigations Office, set up to address the use of lethal force by police in the province has in its few years of existence shown the perils and limitations of such would be oversight efforts. A

KeyboardInterrupt: Interrupted by user

In [159]:
# print(Missing.head())#
# Missing2 = Missing.reset_index(drop=True).set_index(Missing['id_victim']).drop('id_victim',axis=1)
# print(Missing2.head())#
# print(PID['cause_death'])
# print(PID['DEATH CATEGORY'])
print(PID.loc[PID.date=='2016-10-19',['date','MD_match','age','name','id_victim','last_name']])

                 date MD_match   age                        name id_victim  \
date                                                                         
2016-10-19 2016-10-19     None  42.0  Juan Carlos Salvany Sailes    169_GS   
2016-10-19 2016-10-19     None   NaN                 Unspecified     MR_47   

              last_name  
date                     
2016-10-19       Sailes  
2016-10-19  Unspecified  


In [116]:
# Updated['extra_source']
# Updated.to_csv('Compiled/UpdatedLocations.csv')
# print(Missing)

id_victim
MR_225                                                     NaN
MR_210                                                     NaN
MR_221                                                     NaN
MR_195                                                     NaN
MR_190                                                     NaN
MR_189                                                     NaN
MR_179                                                     NaN
MR_176                                                     NaN
MR_175                                                     NaN
MR_168                                                     NaN
MR_157                                                     NaN
MR_156                                                     NaN
MR_153                                                     NaN
MR_145                                                     NaN
MR_138                                                     NaN
MR_136                                       